## Question 1. Install Spark and PySpark
1. Install Spark
2. Run PySpark3. 
Create a local spark sessio
4. Execute spark.versiono5. What's the output?n


In [40]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import concat_ws, year, month, dayofmonth, unix_timestamp

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

spark.version

'3.5.0'

## Question 2. FHV October 2019 partition size

In [13]:
input_path = f'data/raw/fhv/2019/10/'
output_path = f'data/pq/fhv/2019/10/'

df_fhv = (
    spark.read
    .option("header", "true") \
    .option("inferSchema", True) \
    .csv(input_path)
)

(
    df_fhv
    .repartition(6) \
    .write.parquet(output_path, mode='overwrite')
)

In [17]:
!ls -lh ./data/pq/fhv/2019/10/

total 39M
-rw-r--r-- 1 faisal faisal    0 Feb 25 19:55 _SUCCESS
-rw-r--r-- 1 faisal faisal 6.4M Feb 25 19:55 part-00000-43bf6a3e-f177-451a-a402-715f8bf27ab3-c000.snappy.parquet
-rw-r--r-- 1 faisal faisal 6.4M Feb 25 19:55 part-00001-43bf6a3e-f177-451a-a402-715f8bf27ab3-c000.snappy.parquet
-rw-r--r-- 1 faisal faisal 6.4M Feb 25 19:55 part-00002-43bf6a3e-f177-451a-a402-715f8bf27ab3-c000.snappy.parquet
-rw-r--r-- 1 faisal faisal 6.4M Feb 25 19:55 part-00003-43bf6a3e-f177-451a-a402-715f8bf27ab3-c000.snappy.parquet
-rw-r--r-- 1 faisal faisal 6.4M Feb 25 19:55 part-00004-43bf6a3e-f177-451a-a402-715f8bf27ab3-c000.snappy.parquet
-rw-r--r-- 1 faisal faisal 6.4M Feb 25 19:55 part-00005-43bf6a3e-f177-451a-a402-715f8bf27ab3-c000.snappy.parquet


## Question 3. Count records on 15th of October

In [18]:
df_fhv_parquet = (
    spark.read
    .parquet('data/pq/fhv/2019/10/')
)

In [35]:
df_fhv_parquet = (
    df_fhv_parquet
    .withColumn('pickup_date',
                concat_ws('-', 
                          year(df_fhv_parquet.pickup_datetime), 
                          month(df_fhv_parquet.pickup_datetime),
                          dayofmonth(df_fhv_parquet.pickup_datetime)
                         )
               )
)

df_fhv_parquet_count = (
    df_fhv_parquet
    .filter(df_fhv_parquet.pickup_date == '2019-10-15')
    .groupBy('pickup_date')
    .count()
)

In [36]:
df_fhv_parquet_count.show()

+-----------+-----+
|pickup_date|count|
+-----------+-----+
| 2019-10-15|62610|
+-----------+-----+



## Question 4. The longest trip 

In [34]:
df_fhv_parquet.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+
|              B02784|2019-10-01 09:55:38|2019-10-01 10:05:43|          89|          85|   NULL|                  NULL|  2019-10-1|
|              B02429|2019-10-21 04:15:47|2019-10-21 04:36:04|         264|         264|   NULL|                B02429| 2019-10-21|
|              B01482|2019-10-19 12:00:00|2019-10-19 12:20:00|         264|         264|   NULL|                B01482| 2019-10-19|
|              B03015|2019-10-11 14:28:00|2019-10-11 14:32:44|         264|         216|   NULL|                B03015| 2019-10-11|
|              B01529|2019-10-21 18:00:26|2019-10-21 18:07:21|         264| 

In [51]:
# Calculate the difference in seconds between dropOff_datetime and pickup_datetime
df_with_diff = (
    df_fhv_parquet
    .withColumn("trip_duration_seconds", 
                F.unix_timestamp(F.col("dropOff_datetime")) - F.unix_timestamp(F.col("pickup_datetime"))
               )
)

# Convert the difference from seconds to hours
df_with_diff = (
    df_with_diff
    .withColumn("trip_duration_hours", F.col("trip_duration_seconds") / 3600)   
)

# Find the maximum trip duration in hours
df_max_trip_duration_hours = (
    df_with_diff
    .groupBy('pickup_date')
    .agg(F.max("trip_duration_hours").alias('max_trip_duration_hours'))
    .orderBy('max_trip_duration_hours', ascending=False)
)

In [53]:
df_max_trip_duration_hours.show(5)

+-----------+-----------------------+
|pickup_date|max_trip_duration_hours|
+-----------+-----------------------+
| 2019-10-11|               631152.5|
| 2019-10-28|               631152.5|
| 2019-10-31|      87672.44083333333|
|  2019-10-1|      70128.02805555555|
| 2019-10-17|                 8794.0|
+-----------+-----------------------+
only showing top 5 rows



## Question 5. Spark UI port 

Answer: 4040

## Question 6. Least frequent pickup location zone

In [59]:
df_zones = (
    spark.read
    .option("header", "true")
    .option("inferSchema", True)
    .csv('data/lookup/zone/taxi_zone_lookup.csv')
)

In [61]:
df_zones.show(5)
df_zones.printSchema()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [66]:
df_joined = (
    df_fhv_parquet
    .join(df_zones
          , df_fhv_parquet.PUlocationID == df_zones.LocationID
          , how = 'left'
         )
    .groupBy('Zone')
    .agg(F.count('*').alias('Zone_Frequency'))
    .orderBy('Zone_Frequency')
)

In [67]:
df_joined.show(5)

+--------------------+--------------+
|                Zone|Zone_Frequency|
+--------------------+--------------+
|         Jamaica Bay|             1|
|Governor's Island...|             2|
| Green-Wood Cemetery|             5|
|       Broad Channel|             8|
|     Highbridge Park|            14|
+--------------------+--------------+
only showing top 5 rows



In [58]:
# spark.stop()

## Done